## Install required libraries and extensions

In [ ]:
!pip install numpy --user
!pip install PDAL --user
!pip install k3d --user
!jupyter nbextension install --py --user k3d
!jupyter nbextension enable --py --user k3d

## Import necessary libraries

In [ ]:
import k3d
import json
import pdal
import numpy as np
from pathlib import Path

### Specify file path

In [ ]:
file_path = Path("data/autzen.laz")

### PDAL pipeline template

In [ ]:
template = [
    {
        'type': 'readers.las',
        'filename': str(file_path.absolute())
    }
]

### Execute PDAL pipeline

In [ ]:
template = json.dumps(template, indent=4)
pipeline = pdal.Pipeline(template)
pipeline.execute()
arr = pipeline.arrays[0]

### Read necessary attributes then free the variables which  no longer required

In [ ]:
x, y, z, r, g, b = [arr[k] for k in ('X', 'Y', 'Z', 'Red', 'Green', 'Blue')]
del arr
del pipeline

### Prepare colors

In [ ]:
c_grad = (r.astype(np.uint32) * (256**2)) + (g.astype(np.uint32) * 256) + b.astype(np.uint32)

### Translate points to keep scene in first quadrant

In [ ]:
# translate
x -= x.min()
y -= y.min()
z -= z.min()
xyz = np.stack([x, y, z], -1).astype(np.float32)

### View Point Cloud

In [ ]:
plot = k3d.plot(camera_auto_fit=True, camera_fov=100.0, name="Point Cloud Viewer")
plot += k3d.points(xyz, c_grad, point_size=5, shader="flat")
plot.display()